  Gabriel Graells Solé - 205638
# Twitter Scrapper - 2020 USA Elections

The goal of this Notebook is to scrappe Tweets related to the election of USA on 2020. To do so we will do a stream search using Twitter's API and a set of keywords related to this topic. We will use **Tweepy** to easy the work with Tweeters API.

The elections ended the Tuesday, November 3, 2020.

In [1]:
import json
import pandas as pd
import numpy as np
import datetime
import csv
from collections import Counter
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor
import time

from google.colab import drive
drive.mount('/content/drive/')
PATH = '/content/drive/My Drive/Colab Notebooks/IR-WA_FinalProject/data/'

Mounted at /content/drive/


## Connect to the API

In [2]:
# Access token informations 
access_token1 = "1330533182681313288-yMxmNMMhN5pnDKzfRBRCK4XXcpeYvO"
access_token_secret1 = "mNOFxiv6JnjV4ze0EiuhXT4xSDkNAQ04NhKdB5g7uTycx"

consumer_key1 = "sDFQbByaPblJTMYaPzcX09sdv"
consumer_secret1 = "RoMLGdY3hgfyzZbj1U5x4BN5K8gIgk6lUmwJZA4o9ak0gotLlu"

In [3]:
# Connect to Twitter API
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

## Scrape News Accounts

Some accounts can provide more trust-worthy information than others, for example news papers accounts should provide more usefull and trust worthy information than random users. Thus in this section we will retrieve tweets from news paper from the USA and internation press.

We want to **filter tweets per day**. To do so we define a window of days in which we want to select tweets. The elections accorred the 03-11-2020 we will take tweets from a week and two days after and a week previous to election day. Then the range of days is the following **27-10-2020 to 12-11-2020**. 

Filtering is a must, since most of the recent tweets are not related to this topic, they are irrelevant for our search engine. Having a collection which contains only relevant docs regarding USA election topic will increase the recall and reduce search time.

In [7]:
# News twitter account
users = ["@GuardianUS","@CNNPolitics","@nytimes","@washingtonpost"
         ,"@HuffPostPol","@bpolitics"]


# Use attributes
attributes = ["created_at","id", "full_text", "favorite_count","retweet_count","user.id","user.name"]

# Init Dataframe
df_news = pd.DataFrame()

# Dates window
start_date = datetime.datetime(2020, 10, 27, 0,0,0)
end_date = datetime.datetime(2020, 11, 12, 0,0,0)

for user in users:

  # Store tweets
  tweets = []

  # Select tweets within range
  for status in Cursor(api.user_timeline, screen_name = user,tweet_mode="extended", lang = "en").items():
    if status.created_at <= end_date and status.created_at >= start_date:
      tweets.append(status)
    elif status.created_at < start_date:
      break
  
  # Get user time line data in json form
  json_data = [t._json for t in tweets]
  
  # Create df per user
  df = pd.io.json.json_normalize(json_data)

  # Get only needed attributes
  df = df[np.intersect1d(df.columns, attributes)]
 
  # Change created_at datetime format
  df['created_at'] = df['created_at'].apply(lambda x: datetime.datetime.strftime(datetime.datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d'))
  
  # Append to final df
  df_news = df_news.append(df)
  
  # Sleep
  time.sleep(1)

# Reset index 
df_news.index = np.arange(len(df_news))

# Save to csv
df_news.to_csv(f"{PATH}news_Tweets.csv", index = False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


# Scrape Political Agents

We will also scrape data from political agents in the USA. Again the goal is to find entities that are more trust worthy and thus we assume that their tweets would be much more relevant for the queries.

In [ ]:
# List of user names
politicians = ["@JoeBiden","@KamalaHarris","@BernieSanders","@BarackObama", "@realDonaldTrump"]

# List of attributes
attributes = ["created_at","id", "full_text", "favorite_count","retweet_count","user.id","user.name"]

# Final dataframe
df_politicians = pd.DataFrame()

# Dates window
start_date = datetime.datetime(2020, 10, 27, 0,0,0)
end_date = datetime.datetime(2020, 11, 12, 0,0,0)

for politician in politicians:

  # Store tweets
  tweets = []

  # Select tweets within range
  for status in Cursor(api.user_timeline, screen_name = politician, tweet_mode="extended", lang = "eng").items():
    if status.created_at <= end_date and status.created_at >= start_date:
      tweets.append(status)
    elif status.created_at < start_date:
      break
  
  # Get user time line data in json form
  json_data = [t._json for t in tweets]
  
  # Create df per user
  df = pd.io.json.json_normalize(json_data)
  print(politician)
  # Get only needed attributes
  df = df[np.intersect1d(df.columns, attributes)]
 
  # Change created_at datetime format
  df['created_at'] = df['created_at'].apply(lambda x: datetime.datetime.strftime(datetime.datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d'))

  # append to final df
  df_politicians = df_politicians.append(df)

# Reset index
df_politicians.index = np.arange(len(df_politicians))

#Save to csv
df_politicians.to_csv(f"{PATH}politicians_Tweets.csv", index = False)

## Scrape Hashtags

Selecting tweets from a stream it is not the best practice to get relevant data, particularly in this topic. Due the debate on the validity of the elections results many users did not tweet the reality of the elections rather their opinion on what was happening. For example if we issue the query: who won the elections? the resulting docs may be missleading since many Trump voters have posted tweets that do not follow the truth. 

One issue that we are facing is the fact that Twitter Standard API only allows to search for tweets created at a maximum interval of 7 days ago from the request date. Thus our search by hashtag is limited to this constrain.

**IMPORTANT**
Twitter has paramter **filter_level** which is used to rate relevance of tweets. **filter_level** takes values ["low", "medium", "high"]

In [38]:
# Hashtags to search
hashtags = ["#USAElections2020", "#uselections", "#USElections", 
            "#2020election", "#2020Election","#USElectionResults",
            "#vote","#voteblue", "#berniesanders","#donaldtrump"]

# Attributes
attributes = ["created_at","id", "full_text", "favorite_count","retweet_count","user.id","user.name"]

# Init dataframe
df_hashtags = pd.DataFrame()

# Number of tweets per hashtag
count = 200

for h in hashtags:
  tweets = []

  for status in Cursor(api.search, q = h, tweet_mode = "extended", lang = "en").items(count):
    tweets.append(status)

  # Get status data in json form
  json_data = [t._json for t in tweets]

  # Create df per hashtag
  df = pd.io.json.json_normalize(json_data)

  # Get only needed attributes
  df = df[np.intersect1d(df.columns, attributes)]
 
  # Change created_at datetime format
  df['created_at'] = df['created_at'].apply(lambda x: datetime.datetime.strftime(datetime.datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d'))

  # Append to final df
  df_hashtags = df_hashtags.append(df)

# Reset index
df_hashtags.index = np.arange(len(df_hashtags))

# Save to csv
df_hashtags.to_csv(f"{PATH}hashtags_Tweets.csv", index = False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


## Merge DataFrames

In [70]:
# Read df
df_news = pd.read_csv(f"{PATH}news_Tweets.csv")
df_politicians = pd.read_csv(f"{PATH}politicians_Tweets.csv")
df_hashtags = pd.read_csv(f"{PATH}hashtags_Tweets.csv")

# Concatenate df
df_final = pd.concat([df_news,df_politicians,df_hashtags])

# Remove duplicates
df_final = df_final.drop_duplicates ()

# Reset index
df_final.index = np.arange(len(df_final))

# Save in csv
df_final.to_csv(f"{PATH}final_Tweets.csv", index = False)